## This assignment may be worked individually or in pairs. 
## Enter your name/names here:
    

In [2]:
#names here
#Abhishek Dayal
#Nathan Daniel

# Assignment 2: Naive Bayes and KNN classifier

In this assignment you'll implement the Naive Bayes and KNN classifiers to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the same Diabetic Retinopathy data set which was used in the previous assignment on decision trees. The implementation details are up to you but, generally it is a good idea to divide your code up into helper functions.

In [3]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log
from random import shuffle

Read the data from a CSV file. You may choose to store it any any format you wish, like a Pandas dataframe, or any other data structure you'd like.

In [4]:
def get_data(filename):
    data = []
#     your code goes here
    data = pd.read_csv(filename, header=None)
    return data

## Part 1: Naive Bayes Classifier

Naive Bayes (NB) classifier is a simple probabilistic classifier that is based on applying the Bayes' theorem and assumes a strong (naive) independence between features. The Diabetic Retinopathy data set contains both categorical and continuous features. Dealing with categorical features has been discussed in detail in class. Continuous attributes, on the other hand, are more interesting to handle. Most commonly, this is done by assuming normal probability distribution over the feature values or by binning the attribute values in a fixed number of bins. In this assignment you'll be implementing the binning approach. For each continuous attribute, you'll construct 3 equal sized bins. For example, feature 5 ranges from `[1 - 120]` the 3 bins that you'll construct will be `[1 - 40]`, `[41 - 80]`, `[81 - 120]`.

Q1. Implement a Naive Bayes classifier. Measure the accuracy of your classifier using 5-fold cross validation and display the confusion matrix. Also print the precision and recall for class label 1 (patients that have been diagnosed with the disease).

In [5]:
#PREP
# bin continuous attributes (needs to be done with subset of df that is training data)
def bin_df(df):
    bins_dict = dict()
    for c in range(2,18):
        out, bins = pd.cut(df[c], 3, retbins=True, labels=[1,2,3])
    #    bins = list(cut[1])
        bins_dict[c] = list(bins)
    #    out = cut[0]
        df[c] = out

    df.head()
    return (df, bins_dict)

In [43]:
def bin_test(df, bins_dict):
    # bins_dict (K,V) = (feature #, bin threshold list)
    binned_df = df
    for c in range(2,18):
        ii = pd.IntervalIndex.from_breaks(bins_dict[c])

        column_binned = pd.cut(binned_df[c], ii)
        
        column_binned.categories = [1,2,3]
        
#        binned_df[c] = column_binned
        print (column_binned)
        
#    binned_df.head()
    return binned_df

In [7]:
def classify(df, record):
    #CLASSIFY
    # maybe use groupBy?
    # calculate ~probability of 1
        # P(record[0]| class=1) * ... * P(record[18]| class=1) * P(class=1)
    prob1 = df[df[19] == 1].count()[0]/df.shape[0]
    prob0 = df[df[19] == 0].count()[0]/df.shape[0]
#     print(prob1)
    groups = df.groupby(df[19])
    g1 = groups.get_group(1)
    g0 = groups.get_group(0)
    
    print('RECORD\n', record)
    print('G1\n', g1.head())
    for i in range(0,18):
        rval = record[0][i]
        print("rval", rval)
        print("numerator", g1[g1[i] == rval].count()[0])
        print("g1 size", g1.shape[0])
        prob1 *= g1[g1[i] == rval].count()[0] / g1.shape[0]
    # calculate ~probability of 0
        # P(record[0]| class=0) * ... * P(record[18]| class=0) * P(class=0)
    for i in range(0,18):
        rval = record[0][i]
        prob0 *= g0[g0[i] == rval].count()[0] / g0.shape[0]
    # return greater of ~probabilities
    
    return 1 if prob1 > prob0 else 0

In [44]:
# your code goes here
# column 19 is class label

# columns 2 - 17
# Col 2: min - 1 max 130
# Col 3: min - 1 max 132
# Col 4: min - 1 max 120
# Col 5: min - 1 max 105
# Col 6: min - 1 max 97
# Col 7: min - 1 max 89
# Col 8: min - 0.349274 max 403.93910800000003
# Col 9: min - 0.0 max 167.131427
# Col 10: min - 0.0 max 106.07009199999999
# Col 11: min - 0.0 max 59.766121
# Col 12: min - 0.0 max 51.423208
# Col 13: min - 0.0 max 20.098605
# Col 14: min - 0.0 max 5.937799
# Col 15: min - 0.0 max 3.086753
# Col 16: min - 0.367762 max 0.592217
# Col 17: min - 0.05790599999999999 max 0.21919899999999998
# your code goes here

df = get_data('messidor_features.txt')

# TODO: trim off class label and hold separately





#OUTPUT
# 5-fold cross validation

data_len = df.shape[0]
interval = (int)(data_len / 5)

# print(data_len)
# print(interval)


for i in range(0, data_len - interval, interval):

    # partition data into train_set and test_set
    train_set = df[0:i].append(df[i+interval:])
    test_set = df[i:i+interval]
    
    #print(test_set.head())
    
    # get binned df (tuple)
    binned_train_set, bins = bin_df(train_set)
    
    # testing bin_test
    bin_test(test_set, bins)
    
    #print(binned_train_set.head())
    tester_record = pd.DataFrame([[1,   1,  2,  2,  2,  2,  1,  1,  2,  1,  1,  1,  1,  1,  1,  1,  2,  2,  1,  1],
                                 [3,   1,  2,  2,  2,  2,  1,  1,  2,  1,  1,  1,  1,  1,  1,  1,  2,  2,  1,  1]])
#    print(classify(binned_train_set, tester_record.loc(0)))
    # run testing set
    #for record in test_set:
    #    classify(binned_train_set, record)
    # print and factor in new accuracy (store numbers for misplaced testing figures for confusion matrix)

# output confusion matrix

# print precision/recall


0       (0.85, 51.0]
1       (0.85, 51.0]
2      (51.0, 101.0]
3      (51.0, 101.0]
4       (0.85, 51.0]
           ...      
225     (0.85, 51.0]
226     (0.85, 51.0]
227    (51.0, 101.0]
228     (0.85, 51.0]
229    (51.0, 101.0]
Name: 2, Length: 230, dtype: category
Categories (3, interval[float64]): [(0.85, 51.0] < (51.0, 101.0] < (101.0, 151.0]]
0                  (0.869, 44.666666666666664]
1                  (0.869, 44.666666666666664]
2      (44.666666666666664, 88.33333333333333]
3      (44.666666666666664, 88.33333333333333]
4                  (0.869, 44.666666666666664]
                        ...                   
225                (0.869, 44.666666666666664]
226                (0.869, 44.666666666666664]
227    (44.666666666666664, 88.33333333333333]
228                (0.869, 44.666666666666664]
229    (44.666666666666664, 88.33333333333333]
Name: 3, Length: 230, dtype: category
Categories (3, interval[float64]): [(0.869, 44.666666666666664] < (44.666666666666664, 88.333

230    (-0.05431583400000006, 134.8792186666667]
231    (-0.05431583400000006, 134.8792186666667]
232      (134.8792186666667, 269.40916333333337]
233    (-0.05431583400000006, 134.8792186666667]
234    (-0.05431583400000006, 134.8792186666667]
                         ...                    
455    (-0.05431583400000006, 134.8792186666667]
456      (134.8792186666667, 269.40916333333337]
457    (-0.05431583400000006, 134.8792186666667]
458    (-0.05431583400000006, 134.8792186666667]
459    (-0.05431583400000006, 134.8792186666667]
Name: 8, Length: 230, dtype: category
Categories (3, interval[float64]): [(-0.05431583400000006, 134.8792186666667] < (134.8792186666667, 269.40916333333337] < (269.40916333333337, 403.93910800000003]]
230          (-0.167131427, 55.71047566666667]
231          (-0.167131427, 55.71047566666667]
232          (-0.167131427, 55.71047566666667]
233          (-0.167131427, 55.71047566666667]
234          (-0.167131427, 55.71047566666667]
                        

Categories (3, interval[float64]): [(-0.051423208000000005, 17.141069333333334] < (17.141069333333334, 34.28213866666667] < (34.28213866666667, 51.423208]]
460    (-0.020098605, 6.699535]
461    (-0.020098605, 6.699535]
462    (-0.020098605, 6.699535]
463    (-0.020098605, 6.699535]
464    (-0.020098605, 6.699535]
                 ...           
685    (-0.020098605, 6.699535]
686    (-0.020098605, 6.699535]
687    (-0.020098605, 6.699535]
688    (-0.020098605, 6.699535]
689    (-0.020098605, 6.699535]
Name: 13, Length: 230, dtype: category
Categories (3, interval[float64]): [(-0.020098605, 6.699535] < (6.699535, 13.39907] < (13.39907, 20.098605]]
460    (-0.0059377990000000005, 1.9792663333333333]
461    (-0.0059377990000000005, 1.9792663333333333]
462    (-0.0059377990000000005, 1.9792663333333333]
463    (-0.0059377990000000005, 1.9792663333333333]
464    (-0.0059377990000000005, 1.9792663333333333]
                           ...                     
685    (-0.0059377990000000005, 

Categories (3, interval[float64]): [(-0.059766121, 19.92204033333333] < (19.92204033333333, 39.84408066666666] < (39.84408066666666, 59.766121]]
690    (-0.051423208000000005, 17.141069333333334]
691    (-0.051423208000000005, 17.141069333333334]
692    (-0.051423208000000005, 17.141069333333334]
693    (-0.051423208000000005, 17.141069333333334]
694    (-0.051423208000000005, 17.141069333333334]
                          ...                     
915    (-0.051423208000000005, 17.141069333333334]
916    (-0.051423208000000005, 17.141069333333334]
917    (-0.051423208000000005, 17.141069333333334]
918    (-0.051423208000000005, 17.141069333333334]
919    (-0.051423208000000005, 17.141069333333334]
Name: 12, Length: 230, dtype: category
Categories (3, interval[float64]): [(-0.051423208000000005, 17.141069333333334] < (17.141069333333334, 34.28213866666667] < (34.28213866666667, 51.423208]]
690    (-0.020098605, 6.699535]
691    (-0.020098605, 6.699535]
692    (-0.020098605, 6.699535]
693

Categories (3, interval[float64]): [(0.41794649, 0.47495366666666666] < (0.47495366666666666, 0.5317903333333334] < (0.5317903333333334, 0.588627]]
920     (0.11167033333333332, 0.16543466666666665]
921     (0.11167033333333332, 0.16543466666666665]
922     (0.05774470699999999, 0.11167033333333332]
923     (0.05774470699999999, 0.11167033333333332]
924     (0.11167033333333332, 0.16543466666666665]
                           ...                    
1145    (0.05774470699999999, 0.11167033333333332]
1146    (0.11167033333333332, 0.16543466666666665]
1147    (0.11167033333333332, 0.16543466666666665]
1148    (0.11167033333333332, 0.16543466666666665]
1149    (0.05774470699999999, 0.11167033333333332]
Name: 17, Length: 230, dtype: category
Categories (3, interval[float64]): [(0.05774470699999999, 0.11167033333333332] < (0.11167033333333332, 0.16543466666666665] < (0.16543466666666665, 0.21919899999999998]]



## Part 2: K Nearest Neighbor (KNN) Classifier

The KNN classifier consists of two stages:-
- In the training stage, the classifier takes the training data and simply memorizes it
- In the test stage, the classifier compares the test data with the training data and simply returns the maximum occuring label of the k nearest data points.

The distance calculation method is central to the algorithm, typically Euclidean distance is used but other distance metrics like Manhattan distance can also be used. In this assignment you'll be implementing the classifier using the Euclidean distance metric. It is important to note that Euclidean distance is very sensitive to the scaling of different attributes hence, before you can build your classifier you have to normalize the values of each feature in the data set.

Q2. Normalize the dataset so that each feature value lies between `[0-1]`.

In [5]:
# your code goes here


Q3. Build your KNN classifier. 

In [6]:
# your code goes here


Q4. Find the best value of k for this data. Try k ranging from 1 to 10. For each k value, use a 5-fold cross validation to evaluate the accuracy with that k. In each fold of CV, divide your data into a training set and a validation set. Print out the best value of k and the accuracy achieved with that value. Return the best value of k. 

In [7]:
# your code goes here


Q5. Now measure the accuracy of your classifier using 5-fold cross validation. In each fold of this CV, divide your data into a training set and a test set. The training set should get sent through your code for Q4, resulting in a value of k to use. Using that k, calculate an accuracy on the test set. You will average the accuracy over all 5 folds to obtain the final accuracy measurement. Print the accuracy as well as the precision and recall for class label 1 (patients that have been diagnosed with the disease).

In [8]:
# your code goes here
